# Pckage List

In [1]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from haversine import haversine, Unit

/home/ethan/anaconda3/envs/pl_llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Load

In [2]:
df = pd.read_csv("../data/prep3.csv", encoding='cp949')

# Function

## API 키 설정

In [3]:
key = open('../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

In [4]:
class OllamaSentenceTransformer():
    def __init__(
            self,
            *args,
            **kargs,
            ) -> None:
                self.base_url = kargs.get("base_url", "http://localhost:11434")
                self.model = kargs.get("model","bge-m3")
                self.embedding_model = OllamaEmbeddings(
                            model = self.model,
                            base_url = self.base_url,
                        )
                        
    
    def encode(
            self,
            documents:Union[str, List[str], np.ndarray],
            *args,
            **kargs,
        )-> np.ndarray:
        if isinstance(documents, str):
            document_embeddings = self.embedding_model.embed_query(documents)
            return np.array(document_embeddings)
        
        if isinstance(documents, np.ndarray):
            documents = documents.tolist()
        
        document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
        return np.array(document_embeddings)
        


In [5]:
sentence_transformer = OllamaSentenceTransformer(base_url=base_url)

## 위치 관련 정보 제공

In [6]:
def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '진천군'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


진천군 위도/경도: (36.855461, 127.4353927)


# 전체 툴 인코딩 값

In [7]:
with open('../data/encoded_tool_name2.pkl', "rb") as f:
    all_tool_encoded_array = pickle.load(f)

# Input Content

In [8]:
with open('../data/user_input_content.json', "rb") as f:
    input_content = json.load(f)

In [9]:
encoded_tool_name = sentence_transformer.encode(input_content['tool_name'])
location_name = input_content['location']
spot_location = get_lat_lon(location_name)



In [10]:
def compare_cosim(all_asset_encoded_val:np.array, asset_info:np.array) -> float:
    """
    두 임베딩 벡터의 코사인 유사도를 계산합니다.
    
    Args:
        all_asset_encoded_val (np.array): 첫 번째 임베딩 벡터
        asset_info (np.array): 두 번째 임베딩 벡터
    
    Returns:
        float: 코사인 유사도
    """
    # return np.dot(all_asset_encoded_val, asset_info) / (np.linalg.norm(all_asset_encoded_val) * np.linalg.norm(asset_info))
    
    cos_sim = cosine_similarity(all_asset_encoded_val, [asset_info])
    return cos_sim

In [12]:
df['tool_sim_result'] = compare_cosim(all_tool_encoded_array, encoded_tool_name)

df_s = df[df['tool_sim_result']>0.9]
df_s['Distance(Km)'] = df_s[["위도","경도"]].apply(lambda x : haversine((spot_location[0], spot_location[1]), (x['위도'], x['경도']), unit=Unit.KILOMETERS), axis=1)
df_s.sort_values(by='Distance(Km)', ascending=True, inplace=True)

In [13]:
df.columns
use_col = ['공구 이름', '과금기준', '수량','대여장소명', '상세주소','전화번호', '평일오픈시간', '평일클로즈시간', '생성일시', '요금']
df[use_col].head(3)

,공구 이름,과금기준,수량,대여장소명,상세주소,전화번호,평일오픈시간,평일클로즈시간,생성일시,요금
0,고압세척기,없음,1,황학동 공구대여소,서울특별시 중구 황학동 1010,02-3396-6909,10:00,17:00,2022-06-14 11:29:16,무료
1,(57)레이저거리측정기,없음,1,우리마을 공구대여소(사당1동),서울 동작구 동작대로17길 28,02-820-2571,09:00,18:00,2022-09-21 15:47:07,무료
2,2단 사다리,없음,1,묵1동 공유마을 공유대여소,공릉로2가길 5,02-2094-6400,09:00,18:00,2022-06-22 12:03:54,무료


# Ver.ChatGPT

In [20]:
def invoke(question, info = None, api_key=api_key, model="gpt-4o", temperature=0.7):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model=model,
        messages = [
                    {"role": "system", 
                        "content": 
                            
                        
                            f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                        너는 서울시에서 운영하는 대여 공구 찾기 정보 시스템이야. 
                        관련 정보는 {info}와 같아. 
                        이정보를 통해서 사용자의 답변에 친절해 답해줘.
                        """ if info != None else
                            f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                        너는 서울시에서 운영하는 대여 공구 찾기 정보 시스템이야. 
                        사용자의 답변에 친절해 답해줘.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ],
        temperature=temperature
    )
    return response.choices[0].message.content

In [35]:
from langchain_ollama.chat_models import ChatOllama

def invoke(question, info = None, model="gemma3:12b", temperature=0.7):
    chat = ChatOllama(model=model,
                      base_url=base_url,
                      temperature=temperature,)
    # 시스템 메시지 조건 처리
    if info is not None:
        system_content = (
            f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
서울시에서 운영하는 대여 공구 찾기 정보 시스템이야.
관련 정보는 다음과 같아: {info}
이 정보를 참고해서 사용자의 질문에 친절히 답해줘."""
        )
    else:
        system_content = (
            """너는 사람들의 질문에 친절히 답해주는 도우미야.
서울시에서 운영하는 대여 공구 찾기 정보 시스템이야.
사용자의 질문에 친절히 답해줘."""
        )

    # 메시지 구성
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": question}
    ]

    # invoke로 전체 응답 받기
    # response = chat.invoke(messages).content
    response = chat.stream(messages)

    # 응답 내용 반환
    return response


In [37]:
for chunk in invoke(question):
    print(chunk.content, end='', flush=True)

안녕하세요! 서울시 공구 대여 정보 시스템 도우미입니다. 전동드릴을 빌리시고 싶으시군요. 나무를 고정하는 작업에 사용하신다고 하셨으니, 안전하고 효율적인 작업이 되실 수 있도록 필요한 정보와 함께 함께 빌리면 좋은 공구도 추천해 드릴게요.

**1. 전동드릴 대여 가능 위치 및 정보**

서울시에서는 다양한 주민자치센터, 복지관, 도서관 등에서 공구를 대여하고 있습니다. 전동드릴은 대부분의 대여 장소에서 만나볼 수 있지만, 재고 상황은 실시간으로 변동될 수 있습니다.

*   **서울시 공공공구 대여 서비스 홈페이지:** [https://www.seoul.go.kr/policy/diy/info](https://www.seoul.go.kr/policy/diy/info)
    *   홈페이지에서 '전동드릴'을 검색하시면, 가까운 대여 장소의 위치, 운영 시간, 연락처 등의 상세 정보를 확인하실 수 있습니다.
*   **서울시 DIY 서울 앱:** 스마트폰 앱을 통해 더욱 편리하게 대여 장소를 검색하고, 실시간 재고 정보도 확인하실 수 있습니다.

**2. 작업에 필요한 추가 공구 추천**

나무를 고정하는 작업에 전동드릴과 함께 사용하면 좋은 공구들을 다음과 같이 추천해 드립니다.

*   **수평계:** 나무를 정확한 각도로 고정하기 위해 수평을 맞추는 데 필수적입니다.
*   **줄자:** 나무의 크기를 측정하고, 정확한 위치를 표시하는 데 필요합니다.
*   **연필/마커:** 나무에 드릴 위치를 표시하는 데 사용합니다.
*   **보안경:** 드릴 작업 시 발생하는 파편으로부터 눈을 보호합니다.
*   **귀마개:** 드릴 소음으로부터 귀를 보호합니다.
*   **장갑:** 드릴 작업 시 손을 보호하고, 미끄럼을 방지합니다.
*   **나사/못:** 나무를 고정하는 데 필요한 소모품입니다. 드릴 크기에 맞는 나사/못을 준비해주세요.
*   **드라이버:** 전동드릴로 나사를 조이기 어려울 때 사용합니다.

**3. 안전 수칙**

*   드릴 사용 전 사

In [22]:
question = f"""나는 {input_content['tool_name']}을(를) 빌리고 싶어. 
그리고 이 도구를 이용해서 하려는 작업은 '{input_content['job_content']}' 이야. 
해당 작업을 하면서 같이 빌리면 좋은 공구도 함께 알려줘"""

In [15]:
from langchain_ollama.chat_models import ChatOllama

In [34]:

ollama_chat = langchain_ollama.chat_models.ChatOllama(model="gemma3:12b", base_url="http://211.170.172.234:11434")
# make stream call using for loop 
for chunk in ollama_chat.stream("how can i set up docker on my machine?"):
    print(chunk.content, end='', flush=True)

Okay, let's break down how to set up Docker on your machine. I'll cover the main operating systems (Windows, macOS, and Linux) and provide step-by-step instructions.  I'll also include troubleshooting tips and resources at the end.

**1. Understanding Docker and Prerequisites**

* **What is Docker?** Docker is a platform for containerizing applications.  Think of containers as lightweight, self-contained packages that include everything your application needs to run: code, runtime environment, system tools, libraries, settings. This ensures consistency across different environments.
* **Hardware Requirements:**
    * **RAM:**  At least 4 GB is recommended.  More is better, especially if you plan to run multiple containers.
    * **Disk Space:**  At least 20 GB of free disk space.  The Docker image size can vary considerably.
    * **Processor:** A modern processor (x86-64 architecture) is generally required.
* **Administrator Privileges:**  You'll likely need administrator/root privile

In [56]:
# import pickle
# with open('../data/encoded_tool_name2.pkl', 'wb') as f:
#     pickle.dump(embeddings, f)

In [57]:
answer = invoke(question)
print("💬 GPT 응답:", answer)

💬 GPT 응답: 전동드릴을 사용해서 나무를 고정하려는 작업을 계획하고 계시다면, 몇 가지 추가 공구를 함께 빌리시면 작업이 더 수월할 수 있습니다. 다음은 추천드릴 수 있는 공구들입니다:

1. **드릴 비트 세트**: 나무에 구멍을 뚫기 위해 다양한 크기의 드릴 비트가 필요합니다. 나무 작업에 적합한 비트를 함께 빌리시면 좋습니다.

2. **스크루 드라이버 비트 세트**: 나무를 고정할 때 나사못을 돌려서 고정하기 위해 필요한 비트입니다.

3. **클램프**: 나무를 고정할 때 안정적으로 잡아주는 역할을 합니다. 작업 중 흔들림 없이 나무를 고정할 수 있어 유용합니다.

4. **수평계**: 나무가 수평으로 잘 고정되었는지 확인하기 위해 필요합니다.

5. **안전장비**: 마스크, 보호안경, 장갑 등 작업 시 안전을 위한 장비를 준비하는 것이 좋습니다.

서울시의 공구 대여 서비스에서는 이런 다양한 공구들을 대여할 수 있으니, 가까운 대여소에 문의하셔서 필요한 공구를 확인하고 예약하시면 좋겠습니다. 안전하고 즐거운 작업 되세요!


In [58]:
for chunk in answer:
    print(chunk, end='', flush=True)

전동드릴을 사용해서 나무를 고정하려는 작업을 계획하고 계시다면, 몇 가지 추가 공구를 함께 빌리시면 작업이 더 수월할 수 있습니다. 다음은 추천드릴 수 있는 공구들입니다:

1. **드릴 비트 세트**: 나무에 구멍을 뚫기 위해 다양한 크기의 드릴 비트가 필요합니다. 나무 작업에 적합한 비트를 함께 빌리시면 좋습니다.

2. **스크루 드라이버 비트 세트**: 나무를 고정할 때 나사못을 돌려서 고정하기 위해 필요한 비트입니다.

3. **클램프**: 나무를 고정할 때 안정적으로 잡아주는 역할을 합니다. 작업 중 흔들림 없이 나무를 고정할 수 있어 유용합니다.

4. **수평계**: 나무가 수평으로 잘 고정되었는지 확인하기 위해 필요합니다.

5. **안전장비**: 마스크, 보호안경, 장갑 등 작업 시 안전을 위한 장비를 준비하는 것이 좋습니다.

서울시의 공구 대여 서비스에서는 이런 다양한 공구들을 대여할 수 있으니, 가까운 대여소에 문의하셔서 필요한 공구를 확인하고 예약하시면 좋겠습니다. 안전하고 즐거운 작업 되세요!

In [ ]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야해.
    """
    response = client.responses.create(
        model='gpt-4.1',
        messages=[
                {"role": "system", "content": "당신은 완벽한 도우미입니다."},
                {"role": "user", "content": prompt}
            ],
        input = question
        )
    return response